In [7]:
import numpy as np
import pandas as pd
import pickle

test_preprocessed = pd.read_csv('test_preprocessed.csv', index_col = 0)

test_preprocessed
X_test = test_preprocessed.drop('id',axis=1)
X_test = X_test.values

In [8]:
class StackingClassifer():
    def __init__(self, estimators, estimators_second, merge_estimator, ):
        
        self.original_clfs = dict(estimators)
        self.original_clfs_second = dict(estimators_second)

        self.m_clf = merge_estimator
        
        self.clf_dict = defaultdict(list)
        self.clf_dict_second = defaultdict(list)
        self.clfs_index = sorted(self.original_clfs.keys())
        self.clfs_index_second = sorted(self.original_clfs_second.keys())

        
    def fit(self, X,y):
        #リストの初期化
        self.clfs_dict =  defaultdict(list) 
        #交差検証(k=5)
        K = 5
        kf = KFold(n_splits=K, shuffle=True, random_state=1 )

        #indexのリストを作成
        index_list = list(kf.split(X, y))
        
        #第一段の学習
        #第一段の予測を次の特徴量としてリストにする
        merge_feature_list = []
        # 初期の学習器で学習を行う
        count = 0
        for clf_name in self.clfs_index:
            #学習器をセット
            clf_origin = self.original_clfs[clf_name]
            #予測のりすと
            preds_temp_list = []
            #交差検証実施
            for train_index, test_index in index_list:
                #この時点で学習器をコピー
                clf_copy = deepcopy(clf_origin)
                #学習スタート
                clf_copy.fit(X[train_index], y[train_index])
                #予測のリストを格納
                preds_temp_list.extend(
                (clf_copy.predict(X[test_index])).tolist())
#                 print(preds_temp_list)
            
                #学習ずみのモデルを格納
                self.clf_dict[clf_name].append(clf_copy)
            if count == 0:
                merge_feature_list = preds_temp_list
                merge_feature_list  = np.array(merge_feature_list).reshape(-1,1)
                
            else:
                preds_temp_list = np.array(preds_temp_list).reshape(-1,1)
                merge_feature_list = np.concatenate((merge_feature_list, preds_temp_list), axis=1)

        #予測のリストを次の学習の特徴量とする.
#             preds_temp_list = np.array(preds_temp_list)
#             merge_feature_list.append(preds_temp_list)
            count += 1

#         print(len(merge_feature_list))
        merge_feature_list = np.array(merge_feature_list)
        print('第一段階の特徴量のサイズ:{0}'.format(merge_feature_list.shape))
        
#         #Xも特徴量に追加 <--これは削除しても良い？
#         merge_feature_list = np.concatenate((merge_feature_list, X), axis=1)
#         print(merge_feature_list.shape)
#         print()
        
    
        #第二学習フェーズ
        y_merged_second = np.hstack([y[test_index]
                              for _, test_index in index_list])
        
        #リストの初期化
        self.clfs_dict_second =  defaultdict(list) 
        #交差検証(k=5)
        K = 5
        kf = KFold(n_splits=K, shuffle=True, random_state=1 )
        #indexのリストを作成
        index_list = list(kf.split(merge_feature_list, y_merged_second))
        
        merge_feature_list_second = []
        # 初期の学習器で学習を行う
        count = 0
        for clf_name in self.clfs_index_second:
            #学習器をセット
            clf_origin_second = self.original_clfs_second[clf_name]
            #予測のりすと
            preds_temp_list_second = []
            #交差検証実施
            for train_index, test_index in index_list:
                #この時点で学習器をコピー
                clf_copy_second = deepcopy(clf_origin_second)
                #学習スタート
                clf_copy_second.fit(merge_feature_list[train_index], y_merged_second[train_index])
                #予測のリストを格納
                preds_temp_list_second.extend(
                (clf_copy_second.predict(merge_feature_list[test_index])).tolist())
#                 print(preds_temp_list)
            
                #学習ずみのモデルを格納
                self.clf_dict_second[clf_name].append(clf_copy_second)
            if count == 0:
                merge_feature_list_second = preds_temp_list_second
                merge_feature_list_second  = np.array(merge_feature_list_second).reshape(-1,1)
                
            else:
                preds_temp_list_second = np.array(preds_temp_list_second).reshape(-1,1)
                merge_feature_list_second = np.concatenate((merge_feature_list_second, preds_temp_list_second), axis=1)

        #予測のリストを次の学習の特徴量とする.
#             preds_temp_list = np.array(preds_temp_list)
#             merge_feature_list.append(preds_temp_list)
            count += 1

#         print(len(merge_feature_list))
            merge_feature_list_second = np.array(merge_feature_list_second)
#         print(merge_feature_list)

        print('第二段階の特徴量のサイズ:{0}'.format(merge_feature_list_second.shape))
    
        #第三学習フェーズ
        X_merged = merge_feature_list_second
        
        print(X_merged.shape)
        
#         print(X_merged.shape)
        y_merged = np.hstack([y_merged_second[test_index]
                              for _, test_index in index_list])
        print(y_merged.shape)
        
        self.m_clf.fit(X_merged, y_merged)
        
        return self
    
    def predict(self, X):
        #第一段階
        #特徴量のリストを作成
        merge_feature_for_test = []
        # それぞれの学習器で学習
        count = 0
        for clf_name in self.clfs_index:
        #予測のリストを作成
            print(clf_name)
            temp_proba_list = []
            # 各学習済みモデルを引き出す
            for clf in self.clf_dict[clf_name]:
                #予測のリストに各学習器の予測を追加
                temp_proba_list.append(clf.predict(X))
            # 予測を行方向に平均をとる。
            merge_feature_for_test.append(
                np.mean(temp_proba_list, axis=0))
            
        print('(予測)第一段階の特徴量のサイズ:{0}'.format(np.array(merge_feature_for_test).T))
        

            
        #第二段階
        #特徴量のリストを作成
        X_merged_second = np.array(merge_feature_for_test).T
#         #Xも特徴量に追加 <--これは削除しても良い？
#         X_merged_second = np.concatenate((X_merged_second, X), axis=1)
        print(X_merged_second.shape)
        merge_feature_for_test_second = []
        # それぞれの学習器で学習
        count = 0
        for clf_name in self.clfs_index_second:
            print(clf_name)
        #予測のリストを作成
            temp_proba_list_second = []
            # 各学習済みモデルを引き出す
            for clf in self.clf_dict_second[clf_name]:
                #予測のリストに各学習器の予測を追加
                temp_proba_list_second.append(clf.predict(X_merged_second))
            # 予測を行方向に平均をとる。
            merge_feature_for_test_second.append(
                np.mean(temp_proba_list_second, axis=0))
            
        print('(予測)第二段階の特徴量のサイズ:{0}'.format(np.array(merge_feature_for_test_second).T))

            
        # 特徴量リストを
        X_merged = np.array(merge_feature_for_test_second).T
        print(X_merged.shape)
        print(X_merged[:,0])
        X_merged_harmonic_mean = 1/((1/X_merged[:,0]  + 1/X_merged[:, 1]  + 1/X_merged[:,2] ) * (1/3))
        print(X_merged_harmonic_mean)
        print(X_merged_harmonic_mean.shape)
        predict_X = np.average(X_merged, axis=1)
        print(predict_X)      
        print(np.exp(predict_X))



        return X_merged_harmonic_mean
            

In [9]:
with open('model_stcl_develop.pickle', mode='rb') as fp:
    clf_stcl = pickle.load(fp)
print(test_preprocessed['id'].shape)
test_preprocessed['y'] =pd.DataFrame(np.exp(clf_stcl.predict(X_test)))
# print(test_preprocessed['y'].shape)
df_pred_stcl = pd.concat([ test_preprocessed['id'],test_preprocessed['y']], axis=1)
df_pred_stcl[['id','y']].to_csv('pred_by_stcl_develop.csv', index = None, header=None)
df_pred_stcl

(477,)
Bridge
est
lgb
rf
ridge
xgb
(予測)第一段階の特徴量のサイズ:[[10.40632481 10.72672706 10.55370125 10.59716635 10.32650783 10.26381397]
 [ 9.48393451  9.75571471  9.72889507  9.66921597  9.49997709  9.4384222 ]
 [ 9.72779369  9.92973627 10.00913326  9.81449748  9.72430704  9.73356628]
 ...
 [ 9.33859531  9.52364966  9.55401447  9.53315285  9.37737777  9.4602356 ]
 [ 9.55721355  9.74305659  9.73800069  9.74300091  9.55907688  9.55949211]
 [ 9.462757    9.53323301  9.50214376  9.47329092  9.49311898  9.39411926]]
(477, 6)
lgb
ridge
xgb
(予測)第二段階の特徴量のサイズ:[[10.64440384 10.37566769 10.50589371]
 [ 9.75218868  9.62156845  9.68761158]
 [ 9.9446621   9.81125504  9.94444084]
 ...
 [ 9.58300887  9.50064971  9.53417397]
 [ 9.7279876   9.66283154  9.69277   ]
 [ 9.5635114   9.5127274   9.54620838]]
(477, 3)
[10.64440384  9.75218868  9.9446621   9.62157163  9.79817994  9.54919483
 10.19120739  9.41684328 10.07240808 10.63750525  9.5572905   9.74160359
 10.11483793  9.36165569  9.56352921  9.41057933  9.52724

,id,y
0,19075,36589.248277
1,19076,16104.102284
2,19077,19924.764830
3,19078,14724.992307
4,19079,16836.316719
5,19080,13689.088649
6,19081,24566.707445
7,19082,11959.468867
8,19083,22468.776897
9,19084,37199.585478
